### Evaluation of results of images generated by DC-GAN:

* FID: 271
* IS: 2.0
* KID: 0.25 ± 0.01

#### Fréchet Inception Distance (FID)

In [10]:
import os
import numpy as np
from scipy.linalg import sqrtm
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image

def load_and_preprocess_images(img_folder, target_size=(299, 299)):
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    images = []
    
    for img_file in os.listdir(img_folder):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(img_folder, img_file)
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            images.append(img_array)

    images = np.vstack(images)
    features = model.predict(images)
    return features

In [11]:
def calculate_fid(act1, act2):
    # Calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    # Calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)

    # Calculate sqrt of product between covariances
    covmean = sqrtm(sigma1.dot(sigma2))

    # Check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # Calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [ ]:
# Paths to your datasets
folder_x = '/root/group2/generated_glioma_dcgan'  # generated
folder_y = '/root/group2/data_original/Training/glioma'   # original

# Load images
images_x = load_and_preprocess_images(folder_x)
images_y = load_and_preprocess_images(folder_y)

# Calculate FID
fid_score = calculate_fid(images_x, images_y)
print(f'FID score: {fid_score}')

42/42 [==============================] - 25s 564ms/step
FID score: 270.828614790668


## Inception Score (IS)

In [ ]:
# IS for real set

import os
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from scipy.stats import entropy

def load_images_from_folder(folder, target_size=(299, 299)):
    images = []
    for img_file in os.listdir(folder):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, img_file)
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            images.append(img_array)
    images = np.vstack(images)
    return images

def calculate_inception_score(images, inception_model):
    preds = inception_model.predict(images)
    marginals = np.mean(preds, axis=0)
    conditional_entropy = np.mean([entropy(preds[i, :], base=2) for i in range(preds.shape[0])])
    marginal_entropy = entropy(marginals, base=2)
    return np.exp(marginal_entropy - conditional_entropy)

# Load the pretrained Inception model
inception_model = InceptionV3(include_top=True, weights='imagenet')

# Load and preprocess images
folder_x = '/root/group2/generated_glioma_dcgan'
images_x = load_images_from_folder(folder_x)

# Calculate Inception Scores
is_x = calculate_inception_score(images_x, inception_model)
print(f'Inception score for generated images: {is_x}')

4/4 [==============================] - 3s 478ms/step
Inception score for generated images: 2.006554310078951


## Kernel Inception Distance (KID)

In [ ]:
from torch_fidelity import calculate_metrics

generated_images_path = '/root/group2/generated_glioma_dcgan'
real_images_path = '/root/group2/data_original/Training/glioma'

kid_score = calculate_metrics(
    input1=generated_images_path, 
    input2=real_images_path,
    isc=False, 
    fid=False, 
    kid=True,
    kid_subset_size=99
)

# Print all keys in the returned dictionary
print("Keys in returned dictionary:", kid_score.keys())

Creating feature extractor "inception-v3-compat" with features ['2048']
Extracting features from input1
Looking for samples non-recursivelty in "/root/group2/generated_glioma" with extensions png,jpg,jpeg
Found 99 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                       
Extracting features from input2
Looking for samples non-recursivelty in "/root/group2/data_original/Training/glioma" with extensions png,jpg,jpeg
Found 1321 samples, some are lossy-compressed - this may affect metrics
Processing samples                                                           
                                                                                 

Keys in returned dictionary: dict_keys(['kernel_inception_distance_mean', 'kernel_inception_distance_std'])


Kernel Inception Distance: 0.2533878876952047 ± 0.010696999648495974
